## Check the setup and connect to the database

In [ ]:
%run "../01-check_setup.ipynb"

# Choose tables to experiment with

In [ ]:
myconn.get_tables(schema="COLORS")

In [ ]:
spec = 'CSS3'
hdf_colors=myconn.table(f'COLORS_{spec}', schema="COLORS")
print(hdf_colors.select_statement)

In [ ]:
hdf_colors.tail(3).collect()

In [ ]:
reference_color='white'

sql_statement=f'''
    WITH "SINGLE_COLOR" AS (SELECT TO_REAL_VECTOR(ARRAY("Red","Green","Blue")) AS "Ref_Color" FROM ({hdf_colors.select_statement}) WHERE "Color_Name"='{reference_color}')
    SELECT TOP 1000
        "Color_Name",
        "Red",
        "Green",
        "Blue",
        ROUND(L2DISTANCE(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "L2Dist",
        ROUND(COSINE_SIMILARITY(TO_REAL_VECTOR(ARRAY("Red","Green","Blue")), "Ref_Color"), 2) AS "CosSim"
    FROM ({hdf_colors.select_statement}) AS "A", "SINGLE_COLOR"
    WHERE "Color_Name" <> 'black'
    ORDER BY "L2Dist", "CosSim" DESC ;
    '''
#print(sql_statement)

(df_colors := 
    (myconn.sql(sql_statement).collect()
)).__len__()

In [37]:

def color_row(row):
    color_name = row["Color_Name"]
    # Add cell background color based on the color name
    return [f"background-color: {color_name}" for _ in row]

In [ ]:
df_colors.style.apply(color_row, axis=1)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Select 5 random rows
df_sample = df_colors.sample(5, random_state=0)

# df_sample = df_colors[:6]

# Create a 3D scatter plot with Plotly
fig = px.scatter_3d(
    df_sample, 
    x='Red', 
    y='Green', 
    z='Blue', 
    text='Color_Name',
    #color=df_sample.apply(lambda row: f'rgb({row.Red},{row.Green},{row.Blue})', axis=1)
)

# Update layout
fig.update_traces(marker=dict(size=8, opacity=0.8, 
                              color=df_sample[['Red', 'Green', 'Blue']].values / 255))
fig.update_layout(
    title="3D RGB Color Distribution", 
    scene=dict(
        xaxis_title='Red', 
        yaxis_title='Green', 
        zaxis_title='Blue',
        xaxis=dict(range=[0, 256]),
        yaxis=dict(range=[0, 256]),
        zaxis=dict(range=[0, 256])
    ),
    width=800,  # Set width to increase size
    height=600   # Set height to increase size
)

# Show figure
fig.show()
